In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('../Base_de_datos/Clientes.xlsx')
df

,id_cliente,nombre_cliente,email,ciudad,fecha_alta
0,1,Mariana Lopez,mariana.lopez@mail.com,Carlos Paz,2023-01-01
1,2,Nicolas Rojas,nicolas.rojas@mail.com,Carlos Paz,2023-01-02
2,3,Hernan Martinez,hernan.martinez@mail.com,Rio Cuarto,2023-01-03
3,4,Uma Martinez,uma.martinez@mail.com,Carlos Paz,2023-01-04
4,5,Agustina Flores,agustina.flores@mail.com,Cordoba,2023-01-05
...,...,...,...,...,...
95,96,Rocio Gonzalez,rocio.gonzalez@mail.com,Cordoba,2023-04-06
96,97,Uma Alvarez,uma.alvarez@mail.com,Cordoba,2023-04-07
97,98,Camila Castro,camila.castro@mail.com,Cordoba,2023-04-08
98,99,Bruno Molina,bruno.molina@mail.com,Villa Maria,2023-04-09


# **1. Analizar problemas con el dataset**

### **Tipos de datos y valores nulos**

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id_cliente      100 non-null    int64         
 1   nombre_cliente  100 non-null    object        
 2   email           100 non-null    object        
 3   ciudad          100 non-null    object        
 4   fecha_alta      100 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 4.0+ KB


### **Estadísticas generales**

In [4]:
df.describe(include='all')


,id_cliente,nombre_cliente,email,ciudad,fecha_alta
count,100.000000,100,100,100,100
unique,NaN,95,100,6,NaN
top,NaN,Olivia Perez,mariana.lopez@mail.com,Rio Cuarto,NaN
freq,NaN,2,1,23,NaN
mean,50.500000,NaN,NaN,NaN,2023-02-19 12:00:00
min,1.000000,NaN,NaN,NaN,2023-01-01 00:00:00
25%,25.750000,NaN,NaN,NaN,2023-01-25 18:00:00
50%,50.500000,NaN,NaN,NaN,2023-02-19 12:00:00
75%,75.250000,NaN,NaN,NaN,2023-03-16 06:00:00
max,100.000000,NaN,NaN,NaN,2023-04-10 00:00:00


### **Conteo de valores faltantes**

In [5]:
df.isnull().sum()

id_cliente        0
nombre_cliente    0
email             0
ciudad            0
fecha_alta        0
dtype: int64

### **Conteo de filas duplicadas**

In [6]:
df.duplicated().sum()

np.int64(0)

### **Frecuencia por ciudad**

In [7]:
df['ciudad'].value_counts() 

ciudad
Rio Cuarto     23
Alta Gracia    21
Carlos Paz     15
Villa Maria    15
Cordoba        13
Mendiolaza     13
Name: count, dtype: int64

### **Verificación de valores únicos en columnas categóricas**

In [8]:
df['ciudad'].unique()

array(['Carlos Paz', 'Rio Cuarto', 'Cordoba', 'Villa Maria',
       'Alta Gracia', 'Mendiolaza'], dtype=object)

### **Verificación de formatos de email**

In [9]:
df['email'].str.contains('@', na=False).sum()

np.int64(100)

# **2. Limpieza de datos**

### **Eliminación de filas duplicadas**

In [10]:
df = df.drop_duplicates() 

### **Convertir fechas a formato datetime**

In [11]:
df['fecha_alta'] = pd.to_datetime(df['fecha_alta'], errors='coerce') 

### **Quitar espacios y capitalizar**

In [12]:
df['ciudad'] = df['ciudad'].str.strip().str.title()

### **Rellenar valores nulos en 'email' con un valor por defecto**

In [13]:
df['email'] = df['email'].fillna('sin_email@desconocido.com') 

### **Eliminar filas con valores nulos en 'id_cliente' o 'nombre_cliente'**

In [14]:
df = df.dropna(subset=['id_cliente', 'nombre_cliente']) 

# **Estandarización**

### **Asegurar que todos los datos sigan el mismo formato:**
- Nombres con mayúscula inicial
- Correos electrónicos en minúsculas
- Fechas en el mismo formato (YYYY-MM-DD)
- Ciudades escritas igual (“CDMX”, no “cdmx”, “Ciudad de México”, etc.)

In [15]:
df['nombre_cliente'] = df['nombre_cliente'].str.title()
df['email'] = df['email'].str.lower()
df['ciudad'] = df['ciudad'].replace({
    'Cd. De Mexico': 'Ciudad de México',
    'Cdmx': 'Ciudad de México',
    'Mexico City': 'Ciudad de México'
})
df['fecha_alta'] = df['fecha_alta'].dt.strftime('%Y-%m-%d')


# **3. Normalización de datos**

### **Normalización de IDs de cliente**

In [16]:
df['id_cliente'] = pd.to_numeric(df['id_cliente'], errors='coerce')

### **Normalización de nombres de cliente**

In [17]:
df['nombre_cliente'] = df['nombre_cliente'].str.strip().str.replace(r'\s+', ' ', regex=True)

### **Normalización de emails**

In [18]:
df['email'] = df['email'].str.strip().str.lower()
df.drop_duplicates(subset=['email'], inplace=True)

### **Normalización de ciudades (adicional)**

In [19]:
variaciones_adicionales = {
    'Df': 'Ciudad de México',
    'D.F.': 'Ciudad de México',
    'Gdl': 'Guadalajara',
    'Mty': 'Monterrey'
}
df['ciudad'] = df['ciudad'].replace(variaciones_adicionales)

### **Normalización de fechas**

In [20]:
df['fecha_alta'] = pd.to_datetime(df['fecha_alta'], errors='coerce')
df['mes_alta'] = df['fecha_alta'].dt.month

# Se genera la columna 'año_alta' para análisis temporal futuro

# **4. Validación Post-Limpieza y Normalización**

### **Verificar el resultado de la limpieza**

In [21]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id_cliente      100 non-null    int64         
 1   nombre_cliente  100 non-null    object        
 2   email           100 non-null    object        
 3   ciudad          100 non-null    object        
 4   fecha_alta      100 non-null    datetime64[ns]
 5   mes_alta        100 non-null    int32         
dtypes: datetime64[ns](1), int32(1), int64(1), object(3)
memory usage: 4.4+ KB


,id_cliente,nombre_cliente,email,ciudad,fecha_alta,mes_alta
0,1,Mariana Lopez,mariana.lopez@mail.com,Carlos Paz,2023-01-01,1
1,2,Nicolas Rojas,nicolas.rojas@mail.com,Carlos Paz,2023-01-02,1
2,3,Hernan Martinez,hernan.martinez@mail.com,Rio Cuarto,2023-01-03,1
3,4,Uma Martinez,uma.martinez@mail.com,Carlos Paz,2023-01-04,1
4,5,Agustina Flores,agustina.flores@mail.com,Cordoba,2023-01-05,1


### **Comparación antes vs después**

# **5. Exportar datos limpios y normalizados**

In [22]:
df.to_csv('../Base_de_datos_limpia/Clientes_limpio.csv', index=False)